# 图像分类是根据图像的语义信息将不同类别图像区分开来，是计算机视觉中重要的基本问题。


**1、准备数据**

**2、配置网络**

	 （1）定义网络
     
     （2）定义损失函数
     
     （3）定义优化算法
     
**3、训练网络**

**4、模型评估**

**5、模型预测**

In [1]:
#导入必要的包
import zipfile
import os
import random
import paddle
import matplotlib.pyplot as plt
from paddle.nn import MaxPool2D,Conv2D,BatchNorm
from paddle.nn import Linear
import sys
import numpy as np
from PIL import Image
from PIL import ImageEnhance
import paddle
from multiprocessing import cpu_count
import matplotlib.pyplot as plt
import json 

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/rcsetup.py:20: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable, Mapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/colors.py:53: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sized
2021-03-29 11:10:32,094 - INFO - font search path ['/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/mpl-data/fonts/ttf', '/opt/co

**数据集介绍**

台湾电力公司、台湾海洋研究所和垦丁国家公园在2010年10月1日至2013年9月30日期间，在台湾南湾海峡、兰屿岛和胡比湖的水下观景台收集的鱼类图像数据集。

该数据集包括23类鱼种，共27370张鱼的图像。


![](https://ai-studio-static-online.cdn.bcebos.com/b7213f7d65c54b6fae98d57de58e3d2ad29b68abc68140b4b283c615144cd40b)

本实践选取5种鱼类数据作为数据集进行训练，被划分为两个子集，训练集和测试集比例为9:1


# **step1、准备数据：**

**1、生成数据列表**

**2、定义数据提供器 train_r、test_r**

**3、定义train_reader、test_reader**

In [2]:
#解压原始数据集，将fish_image.zip解压至data目录下
src_path="/home/aistudio/data/data14492/fish_image23.zip"
target_path="/home/aistudio/data/fish_image"
if(not os.path.isdir(target_path)):
    z = zipfile.ZipFile(src_path, 'r')
    z.extractall(path=target_path)
    z.close()

In [3]:
#存放所有类别的信息
class_detail = []
#获取所有类别保存的文件夹名称
class_dirs = os.listdir(target_path+"/fish_image")

data_list_path="/home/aistudio/data/"

TRAIN_LIST_PATH=data_list_path + "train.txt"
EVAL_LIST_PATH=data_list_path + "eval.txt"

#每次执行代码，首先清空train.txt和eval.txt
with open(TRAIN_LIST_PATH, 'w') as f: 
    pass
with open(EVAL_LIST_PATH, 'w') as f: 
    pass

In [4]:
#总的图像数量
all_class_images = 0
#存放类别标签
class_label=0
# 设置要生成文件的路径
data_root_path="/home/aistudio/data/fish_image/fish_image"
#存储要写进test.txt和train.txt中的内容
trainer_list=[]
eval_list=[]
#读取每个类别，['fish_01', 'fish_02', 'fish_03']
for class_dir in class_dirs:   
    #每个类别的信息
    class_detail_list = {}
    eval_sum = 0
    trainer_sum = 0
    #统计每个类别有多少张图片
    class_sum = 0
    #获取类别路径 
    path = data_root_path + "/" + class_dir
    # 获取所有图片
    img_paths = os.listdir(path)
    for img_path in img_paths:                                  # 遍历文件夹下的每个图片
        name_path = path + '/' + img_path                       # 每张图片的路径
        if class_sum % 10 == 0:                                 # 每10张图片取一个做验证数据
            eval_sum += 1                                       # test_sum为测试数据的数目
            eval_list.append(name_path + "\t%d" % class_label + "\n")
        else:
            trainer_sum += 1 
            trainer_list.append(name_path + "\t%d" % class_label + "\n")#trainer_sum测试数据的数目
        class_sum += 1                                          #每类图片的数目
        all_class_images += 1                                   #所有类图片的数目
    class_label += 1  
    # 说明的json文件的class_detail数据
    class_detail_list['class_name'] = class_dir             #类别名称，如jiangwen
    class_detail_list['class_label'] = class_label          #类别标签
    class_detail_list['class_eval_images'] = eval_sum       #该类数据的测试集数目
    class_detail_list['class_trainer_images'] = trainer_sum #该类数据的训练集数目
    class_detail.append(class_detail_list)         
    
random.shuffle(eval_list)
with open(data_list_path + "eval.txt", 'a') as f:
    for eval_image in eval_list:
        f.write(eval_image) 
        
random.shuffle(trainer_list)
with open(data_list_path + "train.txt", 'a') as f2:
    for train_image in trainer_list:
        f2.write(train_image) 

# 说明的json文件信息
readjson = {}
readjson['all_class_name'] = data_root_path                  #文件父目录
readjson['all_class_sum'] = class_sum 
readjson['all_class_images'] = all_class_images
readjson['class_detail'] = class_detail
jsons = json.dumps(readjson, sort_keys=True, indent=4, separators=(',', ': '))
with open(data_list_path + "readme.json",'w') as f:
    f.write(jsons)
print ('生成数据列表完成！')

生成数据列表完成！


![](https://ai-studio-static-online.cdn.bcebos.com/9d3504947dca47b7bb0bfd50700d4ff4c804e7c344be496bbdfc0d911e66b58f)

In [5]:
import paddle
import paddle.vision.transforms as T
import numpy as np
from PIL import Image


class FishDataset(paddle.io.Dataset):
    """
    23种鱼数据集类的定义
    """

    def __init__(self, mode='train'):
        """
        初始化函数
        """
        assert mode in ['train', 'eval'], 'mode is one of train, eval.'

        self.data = []

        with open('data/{}.txt'.format(mode)) as f:
            for line in f.readlines():
                info = line.strip().split('\t')

                if len(info) > 0:
                    self.data.append([info[0].strip(), info[1].strip()])

        if mode == 'train':
            self.transforms = T.Compose([
                T.Resize((47,47)),    # 随机裁剪大小
                T.RandomHorizontalFlip(0.5),        # 随机水平翻转
                T.ToTensor(),                       # 数据的格式转换和标准化、 HWC => CHW 
                T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 图像归一化
            ])
        else:
            self.transforms = T.Compose([
                T.Resize((47,47)),                 # 图像大小修改
                T.ToTensor(),                  # 数据的格式转换和标准化 HWC => CHW
                T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])   # 图像归一化
            ])
        
    def __getitem__(self, index):
        """
        根据索引获取单个样本
        """
        image_file, label = self.data[index]
        image = Image.open(image_file)

        if image.mode != 'RGB':
            image = image.convert('RGB')

        image = self.transforms(image)

        return image, np.array(label, dtype='int64')

    def __len__(self):
        """
        获取样本总数
        """
        return len(self.data)

In [6]:
train_dataset=FishDataset(mode='train')
eval_dataset=FishDataset(mode='eval')

# step2、配置网络


![](https://ai-studio-static-online.cdn.bcebos.com/1be52cfb6fc24d61995f9313990827a61bd9eb0d92d04eb39edb990770411875)


**卷积层：执行卷积操作提取底层到高层的特征，发掘出图片“局部特性”；**

**池化层：通过降采样的方式，在不影响图像质量的情况下，压缩图片，减少参数；**

**全连接层：池化完成后，将数据“拍平”，丢到Flatten层，然后把Flatten层的输出放到全连接层里，可采用softmax对其进行分类。**



In [7]:
#定义CNN网络
import paddle
import paddle.nn.functional as F

class MyCNN(paddle.nn.Layer):
    def __init__(self):
        super(MyCNN,self).__init__()
        self.conv1 = Conv2D(in_channels=3, out_channels=20, kernel_size=5,stride=1)        
        self.pool1 = MaxPool2D(kernel_size=2, stride=2)
        self.conv2 = Conv2D(in_channels=20, out_channels=50, kernel_size=5,stride=1)
        self.pool2 = MaxPool2D(kernel_size=2, stride=2)
        self.conv3 = Conv2D(in_channels=50, out_channels=50, kernel_size=5,stride=1)
        self.pool3 = MaxPool2D(kernel_size=2, stride=2)
        self.linear1 = Linear(in_features=200, out_features=23)
    
    def forward(self, input):
        x = self.conv1(input)
        x = F.relu(x)
        x = self.pool1(x)

        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool2(x)

        x = self.conv3(x)
        x = F.relu(x)
        x = self.pool3(x)
        x = paddle.flatten(x, start_axis=1,stop_axis=-1)  
        x = self.linear1(x)
        x=F.softmax(x)
        return x

# step3、训练网络&step4、评估网络


In [8]:
import paddle
from paddle import Model

model= Model(MyCNN())

# 配置模型
model.prepare(paddle.optimizer.Adam(learning_rate=0.0005, parameters=model.parameters()), 
              paddle.nn.CrossEntropyLoss(), 
              paddle.metric.Accuracy())

In [9]:
model.fit(train_dataset, epochs=5, batch_size=256, verbose=1)

The loss value printed in the log is the current step, and the metric is the average value of previous step.
Epoch 1/5


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:77: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  return (isinstance(seq, collections.Sequence) and


step 10/97 [==>...........................] - loss: 2.9924 - acc: 0.1566 - ETA: 57s - 660ms/step

In [15]:
model.evaluate(eval_dataset, batch_size=64,verbose=1)

# step5模型预测

In [10]:
# 进行预测操作
predict_result = model.predict(eval_dataset)

In [11]:
import pandas as pd
data=pd.read_csv('./data/eval.txt',header=None,sep='\t')

In [12]:
# 定义画图方法
from PIL import Image

def show_img(img, predict):
    plt.figure()
    plt.title(predict)
    plt.imshow(img, cmap=plt.cm.binary)
    plt.show()

# 抽样展示
for i in range(10):
    img_path=data[0][i]
    real_class=data[1][i]
    predict_class= np.argmax(predict_result[0][i])
    img=Image.open(img_path)
    title=str(i) +'   '+ 'real_class: ' +str(real_class)+'  '+ 'predict_class: ' + str(predict_class)
    show_img(img, title)

In [13]:
model.save('inference_model', training=False)  # save for inference